<a href="https://colab.research.google.com/github/123nol/AI-projects/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"


In [ ]:


def createModel(train_dataset, test_dataset):
    # Create vectorizer
    vectorizer_layer = TextVectorization(
        max_tokens=10000,
        output_mode="tf-idf"
    )

    # Prepare training data
    category_map = {"ham": 0, "spam": 1}
    X_train = train_dataset['message'].values
    Y_train = train_dataset['category'].map(category_map).values

    # Prepare test data
    X_test = test_dataset['message'].values
    Y_test = test_dataset['category'].map(category_map).values

    # Create TF datasets
    train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=1000).batch(32)
    test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

    # Adapt vectorizer to training data only
    vectorizer_layer.adapt(X_train)

    # Create model
    model = tf.keras.Sequential([
        vectorizer_layer,
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")
    ])

    # Compile model
    model.compile(optimizer="Adam",
                 loss="binary_crossentropy",
                 metrics=['accuracy'])

    # Train model
    history = model.fit(
        train_tf_dataset,
        epochs=20,
        validation_data=test_tf_dataset
    )


    test_loss, test_accuracy = model.evaluate(test_tf_dataset)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return model  # Return both model and vectorizer


train_dataset = pd.read_csv(train_file_path, sep='\t', names=['category', 'message'])
test_dataset = pd.read_csv(test_file_path, sep="\t", names=['category', 'message'])


model = createModel(train_dataset, test_dataset)


In [ ]:
def predicte(model, pred_text):
    # Convert text to tensor
    pred_text = tf.constant([pred_text])

    # Make prediction
    pred = model.predict(pred_text).flatten()

    # Convert prediction to label
    prediction = ["spam" if p > 0.5 else "ham" for p in pred]

    return float(pred[0]),prediction[0]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  a,b=predicte(model,pred_text)



  return [a,b]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
